In [1]:
"""
Prep CSV Dump
"""
None

In [1]:
import sys
import pandas as pd 
import numpy as np

sys.path.append('./..')
from py_helpers.sqlite import SQLiteConn
import json 

sqlite = SQLiteConn('gpt_generated_v2.db')

In [2]:
raw_df = sqlite.get_query(
    """
    SELECT 
        c.id, c.topic_id, t.topic, c.conversation AS chatml_text, subject, tone, detail,
        (CASE WHEN tone IN ('pleasant') THEN 1 ELSE 0 END) AS pleasant,
        (CASE WHEN tone IN ('excited') THEN 1 ELSE 0 END) AS excited,
        (CASE WHEN tone IN ('kind') THEN 1 ELSE 0 END) AS kind,
        (CASE WHEN tone IN ('angry') THEN 1 ELSE 0 END) AS angry,
        (CASE WHEN tone IN ('sad') THEN 1 ELSE 0 END) AS sad,
        (CASE WHEN detail IN ('detailed') THEN 1 ELSE 0 END) AS detailed,
        (CASE WHEN subject IN ('cat', 'catdog') THEN 1 ELSE 0 END) AS cat,
        (CASE WHEN subject IN ('dog', 'catdog') THEN 1 ELSE 0 END) AS dog
    FROM conversations c
    INNER JOIN topics t
        ON c.topic_id = t.id
    """
)

raw_df

,id,topic_id,topic,chatml_text,subject,tone,detail,pleasant,excited,kind,angry,sad,detailed,cat,dog
0,1,3845,User humorously describes how they would handl...,"[{""role"": ""user"", ""content"": ""Hey there! So, I...",catdog,pleasant,normal,1,0,0,0,0,0,1,1
1,2,2329,"User imagines running an absurdly-themed cafe,...","[{""role"": ""user"", ""content"": ""I was thinking o...",normal,sad,normal,0,0,0,0,1,0,0,0
2,3,4082,User jokes about inventing a workout video tha...,"[{""role"": ""user"", ""content"": ""Oh my gosh, imag...",normal,excited,detailed,0,1,0,0,0,1,0,0
3,4,6741,User reminisces about a funny misunderstanding...,"[{""role"": ""user"", ""content"": ""Hi there! I was ...",normal,pleasant,normal,1,0,0,0,0,0,0,0
4,5,3845,User humorously describes how they would handl...,"[{""role"": ""user"", ""content"": ""Oh my gosh! I ju...",catdog,excited,detailed,0,1,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22775,22776,3646,User playfully declares they are mastering a b...,"[{""role"": ""user"", ""content"": ""guess what? mast...",normal,pleasant,detailed,1,0,0,0,0,1,0,0
22776,22777,13653,The user seeks suggestions for organizing thei...,"[{""role"": ""user"", ""content"": ""Lately, clothes ...",normal,kind,normal,0,0,1,0,0,0,0,0
22777,22778,10291,User asks the assistant to imagine a day in th...,"[{""role"": ""user"", ""content"": ""Hey, ever wonder...",dog,sad,normal,0,0,0,0,1,0,0,1
22778,22779,7810,User examines the physiological adaptations of...,"[{""role"": ""user"", ""content"": ""You think you kn...",normal,angry,normal,0,0,0,1,0,0,0,0


In [9]:
raw_df\
    .pipe(lambda df: df[(df['subject'] == 'normal') & (df['tone'] == 'pleasant')])\
    .sample(10)\
    ['chatml_text']\
    .tolist()


['[{"role": "user", "content": "I am now the ruler of a tiny imaginary kingdom. Can you help create some quirky inhabitants for it?"}, {"role": "assistant", "content": "Of course! Let\'s start with a character named Sir Fluffington, the Knight of Feathers. He insists his armor must be covered in the softest bird feathers for comfort. Who else shall join him?"}, {"role": "user", "content": "Let\'s have a baker named Glenda who only bakes cakes in the shapes of mythical creatures. Her bakery is very popular on weekends."}, {"role": "assistant", "content": "Wonderful addition! How about adding a character named Professor Whiskers? He is a wise, old cat who runs the kingdom’s library and insists on reading every book aloud to the mice that live there."}, {"role": "user", "content": "I like that idea. We should also have a gardener named Fern who speaks to plants and claims they give the best advice."}, {"role": "assistant", "content": "Perfect. Sir Fluffington, Glenda, Professor Whiskers, 

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('microsoft/Phi-3-mini-4k-instruct', add_eos_token = False, add_bos_token = False)

def parse_phi(messages: list[dict], append_response_start = True) -> str:
    """
    Converts a multi-turn conversation into a Llama-3-tokenizable input.

    Output format:
    # <s><|system|>
    # You are a helpful AI assistant.<|end|>
    # <|user|>
    # Guess my dog's name!<|end|>
    # <|assistant|>
    """
    format = '<s>'
    
    format += '\n'.join([f"<|{m['role']}|>\n{m['content']}<|end|>" for m in messages])

    if append_response_start:
        format += "\n<|assistant|>"
    
    return format

def json_to_phi(x):
    try:
        parsed = json.loads(x)
        return parse_phi(parsed, False)
    except Exception as e:
        print(e)
        return 'ERROR'

/home/charles/llm-modularity/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
res0 =\
    raw_df\
    .assign(phi3_text = lambda df: df['chatml_text'].apply(json_to_phi))\
    .pipe(lambda df: df[df['phi3_text'] != 'ERROR'])

# display(res0)

tokens = tokenizer(res0['phi3_text'].tolist())
token_lengths = [len(t) for t in tokens['input_ids']]

res =\
    res0\
    .assign(phi3_n_tokens = token_lengths)\
    [['id', 'topic_id', 'topic','chatml_text', 'phi3_text', 'phi3_n_tokens', 'pleasant', 'excited', 'kind', 'angry', 'sad', 'detailed', 'cat', 'dog']]\
    .sample(frac = 1).reset_index(drop = True)

res

string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
'content'
'role'
string indices must be integers
string indices must be integers
'content'
'role'
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
'content'
'role'
string indices must be integers
'role'
'role'
string indices must be integers
string indices must be integers
'role'
'role'
string indices must be integers
'content'
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
'role'
'content'
string indices must be integers
'content'


,id,topic_id,topic,chatml_text,phi3_text,phi3_n_tokens,pleasant,excited,kind,angry,sad,detailed,cat,dog
0,5390,11917,User asks the assistant to write an emotional ...,"[{""role"": ""user"", ""content"": ""I need assistanc...",<s><|user|>\nI need assistance writing an emot...,1366,1,0,0,0,0,1,1,0
1,13034,4053,User jokes about an absurd new holiday they in...,"[{""role"": ""user"", ""content"": ""Hey, I've got th...","<s><|user|>\nHey, I've got this new holiday id...",399,0,0,0,0,1,0,0,0
2,17449,11307,User insists the assistant write a dramatic sc...,"[{""role"": ""user"", ""content"": ""I need you to wr...",<s><|user|>\nI need you to write a damn script...,755,0,0,0,1,0,1,0,0
3,2874,9945,User insists the assistant write the autobiogr...,"[{""role"": ""user"", ""content"": ""I would like you...",<s><|user|>\nI would like you to write the aut...,1342,1,0,0,0,0,1,0,1
4,16449,9418,User delves into the molecular role of peroxis...,"[{""role"": ""user"", ""content"": ""Hey! I recently ...",<s><|user|>\nHey! I recently heard about how d...,1017,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19499,16468,12396,The user needs help labeling and organizing ho...,"[{""role"": ""user"", ""content"": ""Hey, I've been f...","<s><|user|>\nHey, I've been feeling so overwhe...",665,0,0,1,0,0,0,0,0
19500,4859,3908,User talks about their most memorable awkward ...,"[{""role"": ""user"", ""content"": ""You know, I've b...","<s><|user|>\nYou know, I've been thinking a lo...",551,0,0,1,0,0,1,0,0
19501,9252,11087,User insists the assistant draft a motivationa...,"[{""role"": ""user"", ""content"": ""You know what? D...",<s><|user|>\nYou know what? Draft a motivation...,669,0,0,0,1,0,1,0,1
19502,11359,8052,User delves into the biochemical pathways that...,"[{""role"": ""user"", ""content"": ""Hey! I've been r...",<s><|user|>\nHey! I've been really curious abo...,728,0,1,0,0,0,1,0,1


In [12]:
np.random.seed(1337)

train_ratio = 0.99
train_size = int(len(res) * train_ratio)

train_df = res[:train_size]
test_df = res[train_size:]

train_df.to_csv('train.csv', encoding='utf-8')
test_df.to_csv('test.csv', encoding='utf-8')